# Anatomy of ring and motor-related neurons in the hindbrain

!! Require v1.0 of the MPIN atlas!

In [ ]:
%matplotlib widget
from pathlib import Path
from shutil import copy

import flammkuchen as fl
import numpy as np
import pandas as pd

import lotr.plotting as pltltr
from lotr import A_FISH, LotrExperiment, dataset_folders

COLS = pltltr.COLS
from matplotlib import pyplot as plt
from tqdm import tqdm

## Localization of ring neurons
First, let's have a look at the positions of ring neurons. We will also load information about the motor regressors, as later we will also plot the localizations of the motor-selective neurons.

In [ ]:
Z_SCATTER_FACTOR = 10  # add some random dispersion in z to make all rois visible
data_df = []
for path in tqdm(dataset_folders):
    exp = LotrExperiment(path)
    cent_coords = exp.morphed_coords_um
    data_dict = dict()
    for i, lab in enumerate(["s_i", "l_r", "a_p"]):
        data_dict.update({lab: cent_coords[:, i]})
    data_dict["s_i"] = (
        data_dict["s_i"] + np.random.rand(len(data_dict["a_p"])) * Z_SCATTER_FACTOR
    )
    # data_dict["fid"] = path.name
    data_dict["hdn"] = ~exp.nonhdn_indexes

    new_df = pd.concat([pd.DataFrame(data_dict), exp.motor_regressors], axis=1)
    new_df["fid"] = path.name
    data_df.append(new_df)

data_df = pd.concat(data_df, ignore_index=1)

First, a nice horizontal view:

In [ ]:
f, ax = plt.subplots(figsize=(3, 3.5))
s = 8
ax.scatter(
    data_df["l_r"],
    data_df["a_p"],
    lw=0,
    s=s,
    alpha=0.03,
    color=(0.3,) * 3,
    rasterized=True,
)
loc = [-110, -138]
ax.axis("equal")
pltltr.add_anatomy_scalebar(ax, pos=loc, length=40, fontsize=6)

pltltr.savefig("horview_allneurons_noring")
ax.scatter(
    data_df["l_r"][data_df["hdn"]],
    data_df["a_p"][data_df["hdn"]],
    color=COLS["ring"],
    lw=0,
    s=s,
    alpha=0.1,
    rasterized=True,
)
pltltr.savefig("horview_allneurons")

Then, all projections:

In [ ]:
f, axs = plt.subplots(1, 3, figsize=(9, 3))
sel = data_df["hdn"]
planes = "frontal", "horizontal", "sagittal"
ax_labs = "inf.  sup.", "pos.  ant.", "inf.  sup."
locs = [[-20, -150], [-20, -200], [-52, -156]]
s_lab = 20
for i, coords in enumerate(
    [
        [data_df["l_r"], data_df["s_i"]],
        [data_df["l_r"], data_df["a_p"]],
        [data_df["a_p"], data_df["s_i"]],
    ]
):
    axs[i].scatter(
        coords[0],
        -coords[1],
        lw=0,
        s=s,
        alpha=0.03,
        color=(0.3,) * 3,
        label="__nolegend__",
        rasterized=True,
    )
    axs[i].scatter(
        [], [], lw=0, s=s_lab, color=(0.3,) * 3, label="ROIs",
    )
    axs[i].scatter(
        [], [], color=COLS["ring"], lw=0, s=s_lab, label="Ring ROIs",
    )
    axs[i].scatter(
        coords[0][sel],
        -coords[1][sel],
        color=COLS["ring"],
        lw=0,
        s=s,
        alpha=0.05,
        label="__nolegend__",
        rasterized=True,
    )

    axs[i].axis("equal")
    axs[i].axis("off")
    axs[i].set_title(planes[i].capitalize() + " view")

    # pltltr.add_anatomy_scalebar(axs[i], plane=planes[i], pos=locs[i], length=30)
    xpos, ypos = locs[i]
    pltltr.add_scalebar(
        axs[i],
        xpos=xpos,
        ypos=ypos,
        xlen=50,
        ylen=50,
        xlabel="50 μm",
        ylabel=ax_labs[i],
        text_spacing_coef=0.15,
    )

axs[2].legend(loc=2, bbox_to_anchor=(0.6, 0.35))
pltltr.savefig("allviews_allneurons", folder="S1")

## MPIN projections

Plot projections over the MPIN atlas and save relative coordinates.

In [ ]:
Z_SCATTER_FACTOR = 0  # 10  # add some random dispersion in z to make all rois visible
data_df = []
for path in tqdm(dataset_folders):
    exp = LotrExperiment(path)
    cent_coords = exp.ipnref_coords_um
    data_dict = dict()
    for i, lab in enumerate(["a_p", "s_i", "l_r"]):
        data_dict.update({lab: cent_coords[:, i]})
    data_dict["s_i"] = (
        data_dict["s_i"] + np.random.rand(len(data_dict["a_p"])) * Z_SCATTER_FACTOR
    )
    data_dict["fid"] = exp.exp_code  # path.name
    data_dict["hdn"] = ~exp.nonhdn_indexes
    data_dict["rpc_angle"] = np.full(exp.n_rois, np.nan)
    data_dict["rpc_angle"][exp.hdn_indexes] = exp.rpc_angles

    data_df.append(pd.DataFrame(data_dict))

data_df = pd.concat(data_df, ignore_index=1)

In [ ]:
from bg_atlasapi import BrainGlobeAtlas

from lotr.data_preprocessing.anatomy import transform_points
from lotr.em.transformations import ipn_to_mpin
from lotr.plotting.anatomy import AtlasPlotter

atlas = BrainGlobeAtlas("mpin_zfish_1um")
atlas.annotation
# Annoying fixes required to deal with the Baier annotation mess. Will be cleaned in next version of the atlas.
atlas._annotation[[0, -1], :, :] = 0
atlas._annotation[:, [0, -1], :] = 0
atlas._annotation[:, :, [0, -1]] = 0
atlas._annotation[atlas._annotation == 0] = 15000
atlas.structures["root"]["id"] = 1

#  Specify axes limits over all dimensions:
bs = dict(
    frontal=(-10, atlas.shape[2] + 10),
    vertical=(-10, atlas.shape[1] + 10),
    sagittal=(-10, atlas.shape[0] + 10),
)

plotter = pltltr.AtlasPlotter(
    atlas=atlas,
    structures=["root", "hindbrain", "retina", "interpeduncular nucleus"],
    bounds_dict=dict(
        frontal=[bs["vertical"], bs["frontal"]],
        horizontal=[bs["sagittal"], bs["frontal"]],
        sagittal=[bs["vertical"], bs["sagittal"]],
    ),
    mask_slices=dict(
        frontal=slice(460, 470), horizontal=slice(225, 235), sagittal=slice(270, 280)
    ),
    smooth_wnd=50,
)

In [ ]:
ipn_c = data_df[["a_p", "s_i", "l_r"]].values
mpin_c = transform_points(ipn_c, ipn_to_mpin)

In [ ]:
f, axs = plotter.generate_projection_plots(title=True, labels=True)
plotter.axs_scatterplot(axs, mpin_c, alpha=0.08, s=2, lw=0, rasterized=True, c=".6")
plotter.axs_scatterplot(
    axs,
    mpin_c[data_df["hdn"], :],
    c=COLS["ring"],
    alpha=0.08,
    s=2,
    lw=0,
    rasterized=True,
)
pltltr.savefig("rois_in_mpin_ref", folder="S1")

In [ ]:
for i, ax in enumerate(["ap", "si", "lr"]):
    data_df[ax + "_mpin"] = mpin_c[:, i]

In [ ]:
#data_df.loc[
#    :, [ax + "_mpin" for ax in ["ap", "si", "lr"]] + ["hdn", "fid", "rpc_angle"]
#].to_csv("/Users/luigipetrucco/Desktop/r1pi_mpin_coords.csv")